In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn 
import cv2

Using TensorFlow backend.


In [2]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    # Print New Line on Complete
    if iteration == total: 
        print()

In [3]:
class Config(object):
    DEFAULT_VIDEO_PATH = "test.mp4"
    
    # Count frame per prediction
    WINDOW_SIZE = 30
    
    # Interval per last windows frame and prediction frame
    FORECAST = 1
    
    # Convert video frames in gray 
    VIDEO_TO_GRAY = 0
    
    REPLACE_ZERO_TO_INFINITELY_SMALL = 0.0001


In [4]:
class VideoLoader():
    def __init__(self, config=Config):
        self.config = config
        self.video = []
        self.datasets = {}
        
        self.R_min_color = 255
        self.G_min_color = 255
        self.B_min_color = 255
        
        self.R_max_color = 0
        self.G_max_color = 0
        self.B_max_color = 0
        
        self.grey_min_color = 255
        self.grey_max_color = 0
        
    
    def normalize(self, frame_orig):
        frame = frame_orig.copy().astype(float)
        if len(frame.shape) == 3:  # RGB
            if self.R_min_color > self.R_max_color:
                self.R_min_color, self.R_max_color = self.R_max_color, self.R_min_color
            if self.G_min_color > self.G_max_color:
                self.G_min_color, self.G_max_color = self.G_max_color, self.G_min_color
            if self.B_min_color > self.B_max_color:
                self.B_min_color, self.B_max_color = self.B_max_color, self.B_min_color

            frame[:,:,0] = (frame[:,:,0] - self.R_min_color) / ((self.R_max_color - self.R_min_color) or 1)
            frame[:,:,1] = (frame[:,:,1] - self.G_min_color) / ((self.G_max_color - self.G_min_color) or 1)
            frame[:,:,2] = (frame[:,:,2] - self.B_min_color) / ((self.B_max_color - self.B_min_color) or 1)
        else:                      # GREY
            if self.grey_min_color > self.grey_max_color:
                self.grey_min_color, self.grey_max_color = self.grey_max_color, self.grey_min_color
            frame = ((frame - self.R_min_color) / ((self.R_max_color - self.R_min_color) or 1)) or REPLACE_ZERO_TO_INFINITELY_SMALL
        frame[frame == 0] = self.config.REPLACE_ZERO_TO_INFINITELY_SMALL
        return frame
            
    
    def video_to_dataset(self, dataset_name="default", verbose=1):
        if dataset_name in self.datasets.keys():
            del self.datasets[dataset_name]
        
        if not len(self.video):
            raise Exception('Video not loaded!')
    
        min_video_len = self.config.FORECAST + self.config.WINDOW_SIZE
        if len(self.video) < min_video_len:
            raise Exception('Video is short!')
            
        if verbose:
            printProgressBar(0, len(self.video), prefix = 'Progress:', suffix = 'Complete', length = 50)
        self.datasets[dataset_name] = {"X":[], "y":[]}
        windows = []
        for i in range(len(self.video)):
            frame = self.video[i]
            if len(windows) < self.config.WINDOW_SIZE:
                windows.append([])

            for j in range(len(windows)):
                windows[j]
                windows[j].append(self.normalize(frame))
                if len(windows[j]) == min_video_len:
                    self.datasets[dataset_name]["X"].append(windows[j][:-1*self.config.FORECAST])
                    self.datasets[dataset_name]["y"].append(windows[j][-1])
                    windows[j] = windows[j][self.config.WINDOW_SIZE:]
            if verbose:
                printProgressBar(i+1, len(self.video), prefix = 'Progress:', suffix = 'Complete', length = 50)
        return True
            
    def delete_cache_video(self):
        del self.video
        self.video = []
        
        self.R_min_color = 255
        self.G_min_color = 255
        self.B_min_color = 255
        
        self.R_max_color = 0
        self.G_max_color = 0
        self.B_max_color = 0
        
        self.grey_min_color = 255
        self.grey_max_color = 0
        return True
            
    
    def load_gen(self, path=None, save_video=1, to_dataset=0, dataset_name="default", verbose=1, batch_size=None):
        # delete buffer
        self.delete_cache_video()
        
        # set path
        path = path or self.config.DEFAULT_VIDEO_PATH
        if verbose:
            print("Open video path: ", path)
            
        # prepare
        cap = cv2.VideoCapture(path)
        video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if verbose:
            print("Video have: ",video_length, " frame")
        
        if verbose:
            printProgressBar(0, video_length, prefix = 'Progress:', suffix = 'Complete', length = 50)
        
        # start load video
        i = 0
        while(cap.isOpened()):
            # get frame
            ret, frame = cap.read()
            
            # exit point
            if not ret or i > 50:
                break
            
            # to gray
            if self.config.VIDEO_TO_GRAY:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                
                grey_min_color = np.min(frame)
                grey_max_color = np.max(frame)
                
                if self.grey_min_color > grey_min_color:
                    self.grey_min_color = grey_min_color
                if self.grey_max_color > grey_max_color:
                    self.grey_max_color = grey_max_color
            else:
                R_min_color = np.min(frame[:,:,0])
                G_min_color = np.min(frame[:,:,1])
                B_min_color = np.min(frame[:,:,2])

                R_max_color = np.max(frame[:,:,0])
                G_max_color = np.max(frame[:,:,1])
                B_max_color = np.max(frame[:,:,2])
                
                # R
                if self.R_min_color > R_min_color:
                    self.R_min_color = R_min_color
                if self.R_max_color > R_max_color:
                    self.R_max_color = R_max_color
                   
                # G
                if self.G_min_color > G_min_color:
                    self.G_min_color = G_min_color
                if self.G_max_color > G_max_color:
                    self.G_max_color = G_max_color
                   
                # B
                if self.B_min_color > B_min_color:
                    self.B_min_color = B_min_color
                if self.B_max_color > B_max_color:
                    self.B_max_color = B_max_color
            
            
            # save video
            if save_video:
                self.video.append(frame)
            i += 1
            printProgressBar(i, video_length, prefix = 'Progress:', suffix = 'Complete', length = 50)
        if verbose:
            print("Close video path: ", path)
        cap.release()

In [5]:
videoLoader = VideoLoader()

In [6]:
videoLoader.load_gen()

Open video path:  test.mp4
Video have:  1442  frame
Close video path:  test.mp4----------------------------------| 3.5% Complete


In [7]:
videoLoader.video_to_dataset()

RGB


NameError: name 'img' is not defined